In [274]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.spatial.transform import Rotation as R 
import pickle 
import os 
import glob 
from mpl_toolkits.mplot3d import Axes3D 
import random 

In [ ]:
# read in pkl file 
dir_pkl = "./results/data_v1/pkl"

# list all pkl files in the directory 
pkl_files = sorted(glob.glob(os.path.join(dir_pkl, "*.pkl")), key=os.path.getmtime)
 
for i, pkl_file in enumerate(pkl_files): 
    
    # Read the pickle file
    with open(pkl_file, 'rb') as f:
        data = pickle.load(f)

    # unpack data 
    state_hist = data['state_hist'] 
    contact_num = data['contact_num'] 
    contact_geom1 = data['contact_geom1'] 
    contact_geom2 = data['contact_geom2'] 
    contact_dist = data['contact_dist'] 
    contact_pos = data['contact_pos'] 
    contact_frame = data['contact_frame'] 
    ctrl_hist = data['ctrl_hist'] 

    # initialize data structures 
    contact_positions = np.empty((1,3))

    # unpack contact data 
    for j, contact_pos_j in enumerate(contact_pos): 
        contact_positions = np.vstack((contact_positions, contact_pos_j)) 

    # find contact positions 

    # classify geometric element which is in contact 

    # record geometric element pairs in contact for each time step 

    break 



2.1.3


In [325]:
hole_parameters = {
    'R': 3.05, 
    'a': 4.5, 
    'b': 7.0 - np.sqrt(3.05**2 - (4.5/2)**2), 
    'd': 10, 
} 
peg_parameters = {
    'r': 3.0, 
    'e': 2.2, 
    'lp': 29, 
    'hp': 30, 
} 

# function to compute distance between a line segment and point 
def dist_point_to_segment(P0, PA, PB): 
    len_sq = np.linalg.norm(PB-PA)**2  
    if len_sq == 0.0: 
        return np.linalg.norm(P0-PA) 
    d = np.linalg.norm(np.linalg.cross((P0-PA),(P0-PB))) / np.linalg.norm(PB-PA) # see: https://mathworld.wolfram.com/Point-LineDistance3-Dimensional.html 
    dot = np.dot((P0-PA),(PB-PA)) 
    p = dot / len_sq # see: https://stackoverflow.com/questions/849211/shortest-distance-between-a-point-and-a-line-segment
    if p < 0: 
        dist = np.linalg.norm(P0-PA)  
    elif p > 1: 
        dist = np.linalg.norm(P0-PB) 
    else: 
        dist = d  
    return dist 

# function to classify hole point into hole geometry element 
def classify_hole_point(point, hole_parameters, tol=0.1): 
    x = point[0]
    y = point[1] 
    z = point[2] 
    hole_geom_class = [] 

    RR = hole_parameters['R'] 
    a = hole_parameters['a'] 
    b = hole_parameters['b']  
    d = hole_parameters['d']  
    z_h_top = -(d/2 - np.sqrt((d/2)**2 - (a/2)**2))
    z_h_bot = -d + (d/2 - np.sqrt((d/2)**2 - (a/2)**2));  

    # vertices 
    HV1 = np.array([+a/2, -np.sqrt(RR**2 - (a/2)**2), z_h_top])
    HV2 = np.array([-a/2, -np.sqrt(RR**2 - (a/2)**2), z_h_top])
    HV3 = np.array([+a/2, -(b+np.sqrt(RR**2 - (a/2)**2)), z_h_top]) 
    HV4 = np.array([-a/2, -(b+np.sqrt(RR**2 - (a/2)**2)), z_h_top]) 
    HV1_bot = np.array([+a/2, -np.sqrt(RR**2 - (a/2)**2), z_h_bot])
    HV2_bot = np.array([-a/2, -np.sqrt(RR**2 - (a/2)**2), z_h_bot])
    HV3_bot = np.array([+a/2, -(b+np.sqrt(RR**2 - (a/2)**2)), z_h_bot]) 
    HV4_bot = np.array([-a/2, -(b+np.sqrt(RR**2 - (a/2)**2)), z_h_bot]) 

    vertices = [HV1, HV2, HV3, HV4] 
    vertices_class = ['HV1','HV2','HV3','HV4'] 
    for i, vertex in enumerate(vertices): 
        dist = np.linalg.norm(point - vertex) 
        if dist < tol: 
            hole_geom_class.append(vertices_class[i]) 

    # curved edges 
    hole_theta_arc = np.pi - np.arcsin((a/2)/RR) 
    theta = np.arctan2(-x, y) 
    rp = np.sqrt(x**2 + y**2) 

    if x < (d/2):  
        z_curved_edge_theoretical = -(d/2) + np.sqrt((d/2)**2 - x**2) 
        if theta > 0 and theta <= hole_theta_arc and np.abs(rp - RR) < tol and np.abs(z - z_curved_edge_theoretical) < 0.1*tol:  
            hole_geom_class.append('HE1')
        elif theta < 0 and theta >= -hole_theta_arc and np.abs(rp - RR) < tol and np.abs(z - z_curved_edge_theoretical) < 0.1*tol:
            hole_geom_class.append('HE2') 

    # straight edges 
    HE3 = [HV1, HV1_bot]
    HE4 = [HV2, HV2_bot]
    HE5 = [HV3, HV3_bot]
    HE6 = [HV4, HV4_bot]

    # check straight edges 
    straight_edges = [HE3, HE4, HE5, HE6]
    straight_edges_class = ['HE3', 'HE4', 'HE5', 'HE6']  
    for i, straight_edge in enumerate(straight_edges): 
        dist = dist_point_to_segment(point, straight_edge[0], straight_edge[1])  
        if dist < tol: 
            hole_geom_class.append(straight_edges_class[i]) 

    # curved faces 
    if theta > 0 and theta <= hole_theta_arc and np.abs(rp - RR) < tol and z < z_curved_edge_theoretical and z > -d-z_curved_edge_theoretical:  
        hole_geom_class.append('HF1')
    elif theta < 0 and theta >= -hole_theta_arc and np.abs(rp - RR) < tol and z < z_curved_edge_theoretical and z > -d-z_curved_edge_theoretical:
        hole_geom_class.append('HF2')

    # if not hole_geom_class: 
    #     hole_geom_class = 'None' 

    return hole_geom_class 

# function to classify peg point into peg geometry element 
def classify_peg_point(point, peg_parameters, tol=0.1):

    x = point[0]
    y = point[1] 
    z = point[2] 
    peg_geom_class = []  

    r = peg_parameters['r'] 
    e = peg_parameters['e'] 
    lp = peg_parameters['lp']  
    hp = peg_parameters['hp']  
    
    # z_h_top = -(d/2 - np.sqrt((d/2)**2 - (a/2)**2))

    # z_h_bot = -d + (d/2 - np.sqrt((d/2)**2 - (a/2)**2));  

    # define vertices 
    PV1 = np.array([+e, -np.sqrt(r**2 - e**2),  0])
    PV2 = np.array([-e, -np.sqrt(r**2 - e**2), 0])
    PV3 = np.array([+e, -lp, 0]) 
    PV4 = np.array([-e, -lp, 0]) 

    # straight edges 
    PE3 = [PV1, PV3]
    PE4 = [PV2, PV4] 
    
    # check straight edges 
    straight_edges = [PE3, PE4] 
    straight_edges_class = ['PE3', 'PE4']   
    for i, straight_edge in enumerate(straight_edges): 
        dist = dist_point_to_segment(point, straight_edge[0], straight_edge[1])  
        if dist < tol: 
            peg_geom_class.append(straight_edges_class[i]) 
    
    # curved edges 
    peg_theta_arc = np.pi - np.arcsin(e/r) 
    theta = np.arctan2(-x, y) 
    rp = np.sqrt(x**2 + y**2) 
  
    if theta > 0 and theta <= peg_theta_arc and np.abs(rp - r) < tol and np.abs(z-0)<tol:  
        peg_geom_class.append('PE1')
    elif theta < 0 and theta >= -peg_theta_arc and np.abs(rp - r) < tol and np.abs(z-0)<tol: 
        peg_geom_class.append('PE2') 

    # curved faces 
    if theta > 0 and theta <= peg_theta_arc and np.abs(rp - r) < tol and z > 0 and z < hp + tol:  
        peg_geom_class.append('PF1')
    elif theta < 0 and theta >= -peg_theta_arc and np.abs(rp - r) < tol and z > 0 and z < hp + tol:
        peg_geom_class.append('PF2') 

    # flat faces 
    if y > -lp-tol and y < -np.sqrt(r**2 - e**2)+tol and z > -tol and z < hp+tol:  
        if np.abs(x-e)<tol: 
            peg_geom_class.append('PF3') 
        elif np.abs(x+e)<tol: 
            peg_geom_class.append('PF4') 

    # if not peg_geom_class: 
    #     peg_geom_class = 'None' 
    return peg_geom_class 


In [326]:
# verify hole geometry classifier by sampling points in 3D space and plotting classifications 

num_points = 1000000  
margin = 1.0  

# box volume for hole 
x_min = -(hole_parameters['R'] + margin)  
x_max = +(hole_parameters['R'] + margin) 
y_min = -(hole_parameters['R'] + hole_parameters['b'])  
y_max = +(hole_parameters['R'] + margin) 
z_min = -(hole_parameters['d'] + margin) 
z_max = +margin 

x_rand = np.random.rand(num_points) * (x_max - x_min) + x_min 
y_rand = np.random.rand(num_points) * (y_max - y_min) + y_min  
z_rand = np.random.rand(num_points) * (z_max - z_min) + z_min 

points = np.vstack((x_rand,y_rand,z_rand)).transpose() 
point_hole_class = [None] * num_points 
point_peg_class = [None] * num_points 

for i, point in enumerate(points): 
    point_hole_class[i] = classify_hole_point(point, hole_parameters, tol=0.1)  
    point_peg_class[i] = classify_peg_point(point, peg_parameters, tol=0.1) 

In [330]:
# create df 
df_point_class = pd.DataFrame(points, columns=['x','y','z']) 
df_point_class[['Hole','HF1','HF2','HE1','HE2','HE3','HE4','HE5','HE6','HV1','HV2','HV3','HV4','Peg','PF1','PF2','PF3','PF4','PE1','PE2','PE3','PE4']] = False  

hole_classes = ['HF1','HF2','HE1','HE2','HE3','HE4','HE5','HE6','HV1','HV2','HV3','HV4'] 
peg_classes = ['PF1','PF2','PF3','PF4','PE1','PE2','PE3','PE4'] 
for i in range(len(points)): 
    for hole_class in hole_classes: 
        if hole_class in point_hole_class[i]: 
            df_point_class.loc[i,hole_class] = True 
            df_point_class.loc[i,'Hole'] = True 
    for peg_class in peg_classes: 
        if peg_class in point_peg_class[i]: 
            df_point_class.loc[i,peg_class] = True 
            df_point_class.loc[i,'Peg'] = True 

In [332]:
# overlay contact positions on visualization of hole 

# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
palette = plt.cm.tab10  # You can choose other palettes like 'Set1', 'viridis', etc.
num_colors = palette.N  # `N` gives the total number of colors in the palette 

# Plot points
# for hole_class in hole_classes[::-1]: 
#     points_plot = df_point_class[df_point_class[hole_class] == True] 
#     ax.scatter(points_plot['x'], points_plot['y'], points_plot['z'], color=palette(random.randint(0, num_colors - 1)), label=hole_class, s=1)
    
for peg_class in peg_classes[::-1]: 
    points_plot = df_point_class[df_point_class[peg_class] == True] 
    ax.scatter(points_plot['x'], points_plot['y'], points_plot['z'], color=palette(random.randint(0, num_colors - 1)), label=peg_class, s=1)

# Add labels
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis') 
ax.set_zlabel('Z axis')

ax.set_aspect('equal', 'box')  # Ensure equal scaling along both axes

# Show the plot
plt.show() 


